In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import scipy
from scipy.stats import linregress
import matplotlib
import bluepysnap as bp

In [2]:
%matplotlib widget

In [3]:
# Defines paths to the two simulation campaigns
s = '../whiskerFlickSim/original/'
s1 = '../whiskerFlickSim/disconnected/'

In [4]:
def getHistByLayer(path_to_sims,bins=None):

    '''
    Thsi function iterates through all of the trials, and produces a spike time histogram for each layer, separately for excitatory and inhibitory cells. 
    Outputs a matrix of size nTrials x 2 x nLayers x nBins
    '''

    numSims = 10
    numLayers = 6
    numBins = 225
    bins = np.linspace(1975,2200,numBins+1)

    allSpikes = np.zeros([numSims,2,numLayers,numBins])
   
    for sim in range(numSims):

        s = bp.Simulation( path_to_sims + '/'+str(sim)+'/simulation_config.json' )

        c = s.circuit
        layerDF = list(c.nodes.get(group='hex_O1',properties=['layer','synapse_class']))[0][1].loc['S1nonbarrel_neurons']
    
        for layer in np.arange(1,numLayers+1):

            layerIds = list(layerDF[layerDF['layer']==layer].index)


            for classIdx, synClass in enumerate(['EXC','INH']):

                classIds = list(layerDF[layerDF['synapse_class']==synClass].index)

                selectedIds = np.intersect1d(layerIds,classIds)

            
                spikes = s.spikes['S1nonbarrel_neurons'].get(group=selectedIds,t_start=1975,t_stop=2200)

            
                if bins is None:
                    
                    spike_hist,bins = np.histogram(list(spikes.index),bins=numBins)

    
                else:
    
                    spike_hist,_ = np.histogram(list(spikes.index),bins=bins)
            
                allSpikes[sim,classIdx,layer-1] = spike_hist


    return allSpikes, bins

In [5]:
spikes_original, bins = getHistByLayer(s)

In [6]:
spikes_disconnected, bins = getHistByLayer(s1,bins)

In [7]:
bins = (bins[:-1]+bins[1:])/2

In [8]:
def meanAndCI(EEGs):

    '''
    Given a matrix of EEG or spike histograms for each trial, finds the mean and 95% CI over trials
    '''

    mean = np.mean(EEGs,axis=0)

    lowerbound = scipy.stats.t.ppf(0.05,9,mean,np.std(EEGs,axis=0))
    upperbound = scipy.stats.t.ppf(0.95,9,mean,np.std(EEGs,axis=0))

    return mean, lowerbound, upperbound

In [9]:
def getEEGsByTypes(path):

    '''
    Given a path ot a simulation, creates a dictionary of EEG signals for each cell type
    '''

    EEGs = {}

    for file in os.listdir(path+'/pkls'):
        t = file.split('.')[0].split('g')[1]
        
        EEGs[t] = pd.read_pickle(path+'/pkls/'+file).values[19750:22000,0] # For memory reasons, we extract only a portion of the signal

    return EEGs

In [10]:
def getEEGByLayer(path_to_sims):

    '''
    Iterates over all trials, and produces a matrix of the EEG contributions of each layer, for excitatory and inhibitory cells separately
    Outputs a matrix of size nTrials x 2 x nLayers x nTimeSteps
    '''

    duration = 22000-19750

    allEEGList = []

    for sim in range(10):

        s = bp.Simulation( path_to_sims +'/'+str(sim) + '/simulation_config.json' )
        c = s.circuit
        
    
        eegEXC = np.zeros((7,duration))
        eegINH = np.zeros((7,duration))
    
        eegList = [eegEXC, eegINH]

        EEGsByTypes = getEEGsByTypes(path_to_sims +'/'+str(sim))
    
        for mtype in np.sort(np.unique(list(EEGsByTypes.keys()))):
    
            synsDF = list(c.nodes.get(group=mtype,properties=['synapse_class']))[0][1].loc['S1nonbarrel_neurons'].iloc[0].values
    
            if synsDF=='EXC':
                synIdx = 0
            elif synsDF=='INH':
                synIdx = 1
    
            layer = int(mtype.split('_')[0].split('L')[1])
    
            if layer == 2 or layer == 3 or layer == 23:
                layer = 2
    
            eegList[synIdx][layer] += (EEGsByTypes[mtype]-np.mean(EEGsByTypes[mtype][:250])) # Removes baseline offset

            allEEGList.append(eegList)                          
    
    
    return np.array(allEEGList)/10

In [11]:
def getEEGs(path):

    '''
    Given a path to simulations, outputs the EEG from each trial
    Outputs a matrix of size nTrials x nTimeSteps
    '''

    EEGs = list(np.zeros(10))

    for i in range(10):

        for file in os.listdir(path+'/'+str(i)+'/pkls'):

            EEGs[i] += pd.read_pickle(path+'/'+str(i)+'/pkls/'+file).values[19750:22000,0] # For memory reasons, we extract only a portion of the signal

    
    EEGs = np.array(EEGs)
    EEGs -= np.mean(EEGs[:,:250],axis=-1)[:,np.newaxis] # Removes baseline offset

    return EEGs

In [12]:
def getStuff(path):

    EEGs = getEEGs(path)
    mean, lowerbound, upperbound = meanAndCI(EEGs)

    return mean, lowerbound, upperbound

In [13]:
def getByLayer(path):

    EEGs = np.array(getEEGByLayer(path))
    mean, lowerbound, upperbound = meanAndCI(EEGs)

    return mean, lowerbound, upperbound 

In [14]:
meanEEG, lowerboundEEG, upperboundEEG = getStuff(s)

In [15]:
mean_layer, lowerbound_layer, upperbound_layer = getByLayer(s)

/gpfs/bbp.cscs.ch/home/tharayil/bluerecording-dev/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2351: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/gpfs/bbp.cscs.ch/home/tharayil/bluerecording-dev/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2352: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


In [16]:
meanEEG_disconnected, lowerboundEEG_disconnected, upperboundEEG_disconnected = getStuff(s1)

In [ ]:
mean_layer_disconnected, lowerbound_layer_disconnected, upperbound_layer_disconnected = getByLayer(s1)

In [ ]:
mean_spikes,lowerbound_spikes,upperbound_spikes = meanAndCI(spikes_original)
mean_spikes_disconnected,lowerbound_spikes_disconnected,upperbound_spikes_disconnected = meanAndCI(spikes_disconnected)

In [ ]:
time = np.linspace(0,225,2250)

In [ ]:
def getTimeIndices(bins,startTime=0,endTime=225):

    '''
    Given a vector of bins from a spike histogram, finds the time steps corresponding to each bin in an EEG trace sampled at 10 kHz 
    '''

    time = np.arange(startTime,endTime,.1)

    timeIndices = []
    for t in bins[startTime:endTime]:
        index = np.argmin(np.abs(t-time))
        timeIndices.append(index)

    return timeIndices

In [ ]:
def getCorrelationsByLayer(EEGs, spikes,bins=np.arange(225),startTime=0,endTime=225):

    '''
    Given an EEG contribution from each layer, sampled at 10kHz, and a spiking histogram from each layer,
    returns the correlation matrix between the EEG contributions and the spike frequencies
    '''

    timeIndices = getTimeIndices(bins,startTime,endTime)

    correlationMatrix = np.zeros([5,5])
    
    for i in range(5):
    
        for j in range(5):
        
            correlationMatrix[i,j] = scipy.stats.pearsonr(spikes[i][startTime:endTime],EEGs[j][timeIndices]).statistic

    return correlationMatrix

In [ ]:
eegDifference = np.sum(mean_layer,axis=0)-np.sum(mean_layer_disconnected,axis=0) # Difference in the EEG contributions per layer between the connected and the disconnected circuit 


In [ ]:
allSpikes = mean_spikes.sum(axis=0) # Sums spike histograms for excitatory and inhibitory cells to produce one for all cells
# Sums 95% CI  for both histograms
lowerbound_spikes = lowerbound_spikes.sum(axis=0) 
upperbound_spikes=upperbound_spikes.sum(axis=0)


In [ ]:
# Does the same as in the cell above, but for the disconnected circuit

allSpikes_disconnected = mean_spikes_disconnected.sum(axis=0)
lowerbound_spikes_disconnected = lowerbound_spikes_disconnected.sum(axis=0)
upperbound_spikes_disconnected=upperbound_spikes_disconnected.sum(axis=0)


In [ ]:
# Finds correlations between EEG and firing rate, for the overall signal and for specified time windows

In [ ]:
mean_spikes[0].shape

In [ ]:
correlationsExcExc = getCorrelationsByLayer(mean_layer[0,[1,2,4,5,6]],mean_spikes[0])
correlationsExcExcP = getCorrelationsByLayer(mean_layer[0,[1,2,4,5,6]],mean_spikes[0],bins=np.arange(225),startTime=35,endTime=40)
correlationsExcExcN = getCorrelationsByLayer(mean_layer[0,[1,2,4,5,6]],mean_spikes[0],bins=np.arange(225),startTime=40,endTime=45)

correlationsExcExcDiff = getCorrelationsByLayer(eegDifference[[1,2,4,5,6]],mean_spikes[0])
correlationsExcExcDiffP = getCorrelationsByLayer(eegDifference[[1,2,4,5,6]],mean_spikes[0],bins=np.arange(225),startTime=35,endTime=40)
correlationsExcExcDiffN = getCorrelationsByLayer(eegDifference[[1,2,4,5,6]],mean_spikes[0],bins=np.arange(225),startTime=40,endTime=45)
correlationsExcExcDiffL = getCorrelationsByLayer(eegDifference[[1,2,4,5,6]],mean_spikes[0],bins=np.arange(225),startTime=45,endTime=200)


In [ ]:
correlationsInhExc = getCorrelationsByLayer(mean_layer[0,[1,2,4,5,6]],mean_spikes[1])
correlationsInhExcP = getCorrelationsByLayer(mean_layer[0,[1,2,4,5,6]],mean_spikes[1],bins=np.arange(225),startTime=35,endTime=40)
correlationsInhExcN = getCorrelationsByLayer(mean_layer[0,[1,2,4,5,6]],mean_spikes[1],bins=np.arange(225),startTime=40,endTime=45)

correlationsInhExcDiff = getCorrelationsByLayer(eegDifference[[1,2,4,5,6]],mean_spikes[1])
correlationsInhExcDiffP = getCorrelationsByLayer(eegDifference[[1,2,4,5,6]],mean_spikes[1],bins=np.arange(225),startTime=35,endTime=40)
correlationsInhExcDiffN = getCorrelationsByLayer(eegDifference[[1,2,4,5,6]],mean_spikes[1],bins=np.arange(225),startTime=40,endTime=45)
correlationsInhExcDiffL = getCorrelationsByLayer(eegDifference[[1,2,4,5,6]],mean_spikes[1],bins=np.arange(225),startTime=45,endTime=200)


In [ ]:
plotTime = np.linspace(-25,200,2250) # Time vector with the stimulus occurring at 0 ms

### Produces Panel b of Figure 6 in the BlueRecording paper

In [ ]:
from matplotlib.gridspec import GridSpec

fig = plt.figure(figsize=(8,6))

gs = GridSpec(6,3, figure=fig)

ax = fig.add_subplot(gs[0,0])

ax.plot(np.arange(-25,200),np.sum(allSpikes,axis=0),c='r')
ax.plot(np.arange(-25,200),np.sum(allSpikes_disconnected,axis=0),c='b')
ax.fill_between(np.arange(-25,200),np.sum(lowerbound_spikes,axis=0),np.sum(upperbound_spikes,axis=0),color='r',alpha=0.3)
ax.fill_between(np.arange(-25,200),np.sum(lowerbound_spikes_disconnected,axis=0),np.sum(upperbound_spikes_disconnected,axis=0),color='b',alpha=0.3)
ax.set_ylabel('Spikes')
ax.set_xticklabels([])

ax = fig.add_subplot(gs[0,1])
                     
ax.plot(plotTime,meanEEG)
ax.plot(plotTime,meanEEG_disconnected)
ax.fill_between(plotTime,lowerboundEEG,upperboundEEG,color='C0',alpha=0.3)
ax.fill_between(plotTime,lowerboundEEG_disconnected,upperboundEEG_disconnected,color='C1',alpha=0.3)
ax.set_ylabel('EEG (V)')
ax.set_xticklabels([])

ax = fig.add_subplot(gs[0,2])
                     
ax.plot(plotTime,meanEEG-meanEEG_disconnected,c='k')
ax.set_ylabel('EEG\nDifference\n(V)')
ax.set_xticklabels([])

    
ax = fig.add_subplot(gs[1,0])

ax.plot(np.arange(-25,200),allSpikes[0],c='r')  
ax.plot(np.arange(-25,200),allSpikes_disconnected[0],c='b')  

ax.fill_between(np.arange(-25,200),lowerbound_spikes[0],upperbound_spikes[0],color='r',alpha=0.3)
ax.fill_between(np.arange(-25,200),lowerbound_spikes_disconnected[0],upperbound_spikes_disconnected[0],color='b',alpha=0.3)
ax.set_ylabel('Spikes')
ax.set_xticklabels([])


ax = fig.add_subplot(gs[2,0])

ax.plot(np.arange(-25,200),allSpikes[1]+allSpikes[2],c='r')  
ax.plot(np.arange(-25,200),allSpikes_disconnected[1]+allSpikes_disconnected[2],c='b')  

ax.fill_between(np.arange(-25,200),lowerbound_spikes[1]+lowerbound_spikes[2],upperbound_spikes[1]+upperbound_spikes[2],color='r',alpha=0.3)
ax.fill_between(np.arange(-25,200),lowerbound_spikes_disconnected[1]+lowerbound_spikes_disconnected[2],upperbound_spikes_disconnected[1]+upperbound_spikes_disconnected[2],color='b',alpha=0.3)
ax.set_ylabel('Spikes')
ax.set_xticklabels([])


ax = fig.add_subplot(gs[3,0])

ax.plot(np.arange(-25,200),allSpikes[3],c='r')  
ax.plot(np.arange(-25,200),allSpikes_disconnected[3],c='b')  

ax.fill_between(np.arange(-25,200),lowerbound_spikes[3],upperbound_spikes[3],color='r',alpha=0.3)
ax.fill_between(np.arange(-25,200),lowerbound_spikes_disconnected[3],upperbound_spikes_disconnected[3],color='b',alpha=0.3)
ax.set_ylabel('Spikes')
ax.set_xticklabels([])


ax = fig.add_subplot(gs[4,0])

ax.plot(np.arange(-25,200),allSpikes[4],c='r')  
ax.plot(np.arange(-25,200),allSpikes_disconnected[4],c='b')  

ax.fill_between(np.arange(-25,200),lowerbound_spikes[4],upperbound_spikes[4],color='r',alpha=0.3)
ax.fill_between(np.arange(-25,200),lowerbound_spikes_disconnected[4],upperbound_spikes_disconnected[4],color='b',alpha=0.3)
ax.set_ylabel('Spikes')
ax.set_xticklabels([])

ax = fig.add_subplot(gs[5,0])

ax.plot(np.arange(-25,200),allSpikes[5],c='r')  
ax.plot(np.arange(-25,200),allSpikes_disconnected[5],c='b')  

ax.fill_between(np.arange(-25,200),lowerbound_spikes[5],upperbound_spikes[5],color='r',alpha=0.3)
ax.fill_between(np.arange(-25,200),lowerbound_spikes_disconnected[5],upperbound_spikes_disconnected[5],color='b',alpha=0.3)
ax.set_xlabel('Time (ms)')
ax.set_ylabel('Spikes')

ax0 = fig.add_subplot(gs[1,1])
ax1 = fig.add_subplot(gs[2,1])
ax2 = fig.add_subplot(gs[3,1])
ax3 = fig.add_subplot(gs[4,1])
ax4 = fig.add_subplot(gs[5,1])

axes = [ax0,ax1,ax2,ax3,ax4]


for i, layer in enumerate([1,2,4,5,6]):

    
    ax = axes[i]
    ax.plot(plotTime,np.sum(mean_layer[:,layer],axis=0))  
    ax.plot(plotTime,np.sum(mean_layer_disconnected[:,layer],axis=0))  
    ax.fill_between(plotTime,np.sum(lowerbound_layer[:,layer],axis=0),np.sum(upperbound_layer[:,layer],axis=0),color='C0',alpha=0.3)
    ax.fill_between(plotTime,np.sum(lowerbound_layer_disconnected[:,layer],axis=0),np.sum(upperbound_layer_disconnected[:,layer],axis=0),color='C1',alpha=0.3)

    if layer == 6:
        ax.set_xlabel('Time (ms)')
    else:
        ax.set_xticklabels([])
    ax.set_ylabel('EEG (V)')

ax0 = fig.add_subplot(gs[1,2])
ax1 = fig.add_subplot(gs[2,2])
ax2 = fig.add_subplot(gs[3,2])
ax3 = fig.add_subplot(gs[4,2])
ax4 = fig.add_subplot(gs[5,2])

axes = [ax0,ax1,ax2,ax3,ax4]


for i, layer in enumerate([1,2,4,5,6]):

    
    ax = axes[i]
    ax.plot(plotTime,np.sum(mean_layer[:,layer],axis=0)-np.sum(mean_layer_disconnected[:,layer],axis=0),c='k') 
    if layer == 6:
        ax.set_xlabel('Time (ms)')
    else:
        ax.set_xticklabels([])
    ax.set_ylabel('EEG\nDifference\n(V)')

    # ax.set_xlim([0,35])


plt.tight_layout()
# plt.show()
plt.savefig('figures/whiskerflick_eeg.png',dpi=300)

### Produces Panel c of Figure 6 in the BlueRecording paper

In [ ]:
fig = plt.figure(figsize=(8,4))

gs = GridSpec(2,4, figure=fig)

ax = fig.add_subplot(gs[0,0])
ax.imshow(correlationsExcExcDiff,cmap='bwr',vmin=-1,vmax=1)
plt.yticks(np.arange(5),labels=['1','2/3','4','5','6'])
plt.xticks([])

ax = fig.add_subplot(gs[0,1])
ax.imshow(correlationsExcExc)
ax.imshow(correlationsExcExcDiffP,cmap='bwr',vmin=-1,vmax=1)
plt.yticks([])
plt.xticks([])

ax = fig.add_subplot(gs[0,2])
ax.imshow(correlationsExcExc)
ax.imshow(correlationsExcExcDiffN,cmap='bwr',vmin=-1,vmax=1)
plt.xticks([])
plt.yticks([])


ax = fig.add_subplot(gs[0,3])
ax.imshow(correlationsExcExcDiffL,cmap='bwr',vmin=-1,vmax=1)
plt.yticks([])
plt.xticks([])


plt.yticks([])
plt.xticks([])

plt.yticks([])
plt.xticks([])

ax = fig.add_subplot(gs[1,0])
ax.imshow(correlationsInhExcDiff,cmap='bwr',vmin=-1,vmax=1)
plt.xticks(np.arange(5),labels=['1','2/3','4','5','6'])
plt.yticks(np.arange(5),labels=['1','2/3','4','5','6'])

ax = fig.add_subplot(gs[1,1])
ax.imshow(correlationsInhExcDiffP,cmap='bwr',vmin=-1,vmax=1)
plt.xticks(np.arange(5),labels=['1','2/3','4','5','6'])
plt.yticks([])

ax = fig.add_subplot(gs[1,2])
ax.imshow(correlationsInhExcDiffN,cmap='bwr',vmin=-1,vmax=1)
plt.xticks(np.arange(5),labels=['1','2/3','4','5','6'])
plt.yticks([])

ax = fig.add_subplot(gs[1,3])
ax.imshow(correlationsInhExcDiffL,cmap='bwr',vmin=-1,vmax=1)
plt.xticks(np.arange(5),labels=['1','2/3','4','5','6'])
plt.yticks([])

plt.tight_layout()
plt.savefig('figures/whiskerflick_correlations.png',dpi=300)